Isthmus / Mitchellpkt

March 2020

## Import libraries

In [15]:
import numpy as np
import isthmuslib as isli
import matplotlib as pyplot 
import random as random

## Parameters

In [16]:
fractional_difficulty = 0.999

## Mimic a PoW iteration

In [22]:
def hash_attempt_boolean(fractional_difficulty):
    random_num = random.random()
    if random_num >= fractional_difficulty:
        return True
    else:
        return False

## Define worker

In [79]:
class Worker:
    def __init__(self, hashes_per_second=100):
        self.hashes_per_second = hashes_per_second
        self.nonce = 0
        self.found_block_yet = False
        
    def reset(self):
        self.nonce = 0
        self.found_block_yet = False
        
    def print(self):
        print('---------------')
        print('Worker hashrate: ' + str(self.hashes_per_second) + ' H/s')
        print('Current nonce: ' + str(self.nonce))
        print('Found block yet: ' + str(self.found_block_yet))

    def single_hash(self, fractional_difficulty, verbose = False):
        self.nonce = self.nonce + 1
        attempt_result = hash_attempt_boolean(fractional_difficulty)
        if attempt_result:
            self.found_block_yet = True
        return attempt_result
    
    def multiple_hash(self, number_of_hashes, fractional_difficulty, verbose = False):
        iteration = 0
        while (self.found_block_yet == False)& (iteration <= number_of_hashes):
            self.single_hash(fractional_difficulty)
            if verbose:
                print('--------------')
                print('Nonce: ' + str (self.nonce))
                print('Step iteration: ' + str(iteration) + ' of ' + str(number_of_hashes))
                print('Successful? ' + str(self.found_block_yet))
            iteration += 1
        return self.found_block_yet
            
    def hash_n_seconds(self, number_of_seconds, fractional_difficulty, verbose = False):
        number_of_hashes = number_of_seconds * hashes_per_second
        multiple_hash(number_of_hashes, fractional_difficulty, verbose = verbose)
        return self.found_block_yet

## Test a single simulation

In [80]:
W = Worker(hashes_per_second=400)

In [81]:
W.reset()

In [84]:
W.multiple_hash(5, 0.9, verbose=True)

--------------
Nonce: 7
Step iteration: 0 of 5
Successful? False
--------------
Nonce: 8
Step iteration: 1 of 5
Successful? True


True